# Path Visibility in Sample Images

In [1]:
import sys
from pathlib import Path
from typing import List

import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

from outdoorar.constants import RESOURCES_DIR, CAMERAS_DIR, OUTPUT_DIR, VISIBILITY_DIR, FIGURES_DIR
from outdoorar.rendering import get_image_coordinates, is_inside_image
from outdoorar.sphere_sampling import get_spherical_coordinates
from outdoorar.visibility import Visibility, Vertex, from_json, calculate_visibility

sys.path.insert(0, '..')

## Input data

In [3]:
captured_images = RESOURCES_DIR.joinpath('capturedImages')
cameras_sfm = CAMERAS_DIR.joinpath('cameras.sfm')

In [4]:
n_range = [2,4,8,16,32]

## Functions

In [5]:
def read_polylines(n, parent_visibility_folder=VISIBILITY_DIR):
    
    visibility_folder = parent_visibility_folder.joinpath(f'n_{n}')
    blue_visibility = from_json(visibility_folder.joinpath('BluePolyline.json'))
    red_visibility = from_json(visibility_folder.joinpath('RedPolyline.json'))
    yellow_visibility = from_json(visibility_folder.joinpath('YellowPolyline.json'))
    green_visibility = from_json(visibility_folder.joinpath('GreenPolyline.json'))

    polylines = [
        (green_visibility, (0, 1, 0.5)), 
        (yellow_visibility, (1, 1, 0)),
        (red_visibility, (1, 0, 0)),
        (blue_visibility, (0, 170/255, 1)),
    ]
    return polylines

In [6]:
def get_or_create_output_directory(n, parent_output_directory=OUTPUT_DIR):
    output_directory = parent_output_directory.joinpath(f'n_{n}')
    output_directory.mkdir(exist_ok=True, parents=True)
    return output_directory

In [7]:
def polyline_to_matrix(visibility: Visibility) -> np.ndarray:
    return np.array([[v.x, v.y, v.z] for v in visibility.vertices])

In [8]:
def clip(coords: np.ndarray, image_width: int, image_height: int) -> np.ndarray:
    return np.maximum(np.minimum(coords, np.array([[image_width], [image_height], [1]])), 0)

In [9]:
def plot_polyline(image_coords, node_visibility, edges, color, image_width, image_height):
    if not np.any(node_visibility):
        return 
    plt.scatter(
        x=image_coords[:, node_visibility][0], 
        y=image_coords[:, node_visibility][1], 
        facecolors=color,
        edgecolors=color
    )
    for edge in edges:
        v1_visible = node_visibility[edge.vertex1]
        v2_visible = node_visibility[edge.vertex2]
        coords = clip(image_coords[:, [edge.vertex1, edge.vertex2]], image_width, image_height)
        if v1_visible and v2_visible:
            plt.plot(coords[0], coords[1], '-', c=color)
        elif v1_visible or v2_visible:
            plt.plot(coords[0], coords[1], '--', c=color)


## Camera information

In [10]:
cameras = json.load(cameras_sfm.open('r'))

In [11]:
cameras.keys()

dict_keys(['version', 'featuresFolders', 'matchesFolders', 'views', 'intrinsics', 'poses'])

### Camera intrinsic matrix

In [12]:
intrinsic = cameras['intrinsics'][0]

In [13]:
K = np.array([
    [float(intrinsic["pxFocalLength"]), 0, float(intrinsic["principalPoint"][0]), 0],
    [0, float(intrinsic["pxFocalLength"]), float(intrinsic["principalPoint"][1]), 0],
    [0, 0, 1, 0]
])

In [14]:
K

array([[3.00185248e+03, 0.00000000e+00, 2.00053931e+03, 0.00000000e+00],
       [0.00000000e+00, 3.00185248e+03, 1.49549277e+03, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00]])

### Sample images

In [15]:
views = {view['poseId'] : {
    'imgName': view['path'][view['path'].rfind('/')+1:].upper(),
    'width': int(view['width']),
    'height': int(view['height'])
} for view in cameras['views']}    

### Prepare output dataframe with visibility information

In [16]:
def get_output_dataframe(polylines, views):
    annotations_info: list[tuple[str, str]] = []

    for curr_polyline, curr_color in polylines:
        poly_name = curr_polyline.name
        num_vertices = len(curr_polyline.vertices)
        annotations_info.extend(list(zip([poly_name] * num_vertices, [str(x) for x in range(num_vertices)])))
        
    images_index = [view['imgName'] for view in views.values()]
    
    return pd.DataFrame(
        data=0, 
        columns=pd.MultiIndex.from_tuples(annotations_info), 
        index=images_index, 
        dtype=int,
    )

### Draw visible annotations in the image

In [31]:
for n in n_range:
    polylines = read_polylines(n)
    output_directory = get_or_create_output_directory(n)
    results_df = get_output_dataframe(polylines, views)
    
    for pose_obj in tqdm(cameras['poses'], desc=f'n={n}'):
        
        pose = pose_obj['pose']['transform']
        view = views[pose_obj['poseId']]
        img_name = view['imgName']
        im = plt.imread(captured_images.joinpath(img_name))
        fig, ax = plt.subplots(figsize=(16,12))
        implot = ax.imshow(im)
        plt.axis('off')

        R = np.array([float(x) for x in pose["rotation"]]).reshape((3,3), order='F')
        C = np.array([[float(x)] for x in pose["center"]])
        T = - np.matmul(R, C)
        M = np.vstack((np.hstack((R, T)), np.array([0, 0, 0, 1])))
        eye = [float(x) for x in pose["center"]]  

        image_width, image_height = view['width'], view['height']

        for curr_polyline, curr_color in polylines:
            
            polyline_matrix = polyline_to_matrix(curr_polyline)
            curr_image_coords = get_image_coordinates(polyline_matrix, K, M)
            curr_is_visible = np.logical_and(
                calculate_visibility(curr_polyline.vertices, eye),
                is_inside_image(curr_image_coords, image_width, image_height)
            )
            results_df.loc[img_name, results_df.columns.get_level_values(0)==curr_polyline.name] = curr_is_visible.astype(int)
            plot_polyline(curr_image_coords, curr_is_visible, curr_polyline.edges, curr_color, image_width, image_height)
        plt.savefig(output_directory.joinpath(img_name), bbox_inches='tight')
        plt.close()
    results_df.to_csv(output_directory.joinpath('visibility.csv'))


n=32: 100%|████████████████████████████████████████████████████████████████████████████| 77/77 [02:21<00:00,  1.83s/it]


### Compare to ground truth

In [17]:
ground_truth_file_path = RESOURCES_DIR.joinpath('ground_truth.csv')
gt_df = pd.read_csv(ground_truth_file_path, header=[0,1], index_col=0)

In [18]:
gt_df

Polyline                BluePolyline          GreenPolyline                  
VertexIdx                          0  1  2  3             0  1  2  3  4  5   
IMG_20230320_153414.JPG            0  0  0  0             1  1  1  1  0  0  \
IMG_20230320_154334.JPG            1  1  1  1             0  0  0  0  0  0   
IMG_20230320_154133.JPG            1  1  1  1             0  0  0  0  0  0   
IMG_20230320_154002.JPG            1  1  1  1             0  0  0  0  0  0   
IMG_20230320_153708.JPG            0  0  0  0             0  1  1  1  1  1   
...                              ... .. .. ..           ... .. .. .. .. ..   
IMG_20230320_152252.JPG            1  1  0  1             0  0  0  0  0  1   
IMG_20230320_152221.JPG            1  1  0  1             0  0  0  0  0  0   
IMG_20230320_154159.JPG            0  0  0  0             0  0  0  0  0  0   
IMG_20230320_152801.JPG            0  0  0  0             1  1  1  1  1  1   
IMG_20230320_153614.JPG            0  0  0  0             0  0  0  0  0  0   

Polyline                 ... YellowPolyline                             
VertexIdx                ...              3  4  5  6  7  8  9 10 11 12  
IMG_20230320_153414.JPG  ...              1  0  0  0  0  1  1  1  0  0  
IMG_20230320_154334.JPG  ...              0  0  0  0  0  0  0  0  0  0  
IMG_20230320_154133.JPG  ...              0  0  0  0  0  0  0  0  0  0  
IMG_20230320_154002.JPG  ...              0  0  0  0  0  0  0  1  1  0  
IMG_20230320_153708.JPG  ...              0  0  1  1  0  1  1  1  1  1  
...                      ...            ... .. .. .. .. .. .. .. .. ..  
IMG_20230320_152252.JPG  ...              1  0  1  1  0  1  0  1  1  1  
IMG_20230320_152221.JPG  ...              1  0  1  1  0  1  0  1  1  1  
IMG_20230320_154159.JPG  ...              0  0  0  0  0  0  0  0  0  0  
IMG_20230320_152801.JPG  ...              1  0  1  1  0  1  1  1  0  1  
IMG_20230320_153614.JPG  ...              0  0  0  0  0  0  0  1  0  0  

[77 rows x 34 columns]

In [19]:
def get_scores(gt_df, results_df, cols):
    tp = np.logical_and(gt_df == 1, results_df[cols] == 1).sum().sum()
    tn = np.logical_and(gt_df == 0, results_df[cols] == 0).sum().sum()
    fp = np.logical_and(gt_df == 0, results_df[cols] == 1).sum().sum()
    fn = np.logical_and(gt_df == 1, results_df[cols] == 0).sum().sum()
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = (2*tp) / (2*tp + fp + fn)
    return accuracy, precision, recall, f1

cols = gt_df.columns

scores = pd.DataFrame(
    data=np.nan, 
    columns=['Accuracy', 'Precision', 'Recall', 'F1-Score'], 
    index=pd.Index(n_range, name='Number of viewpoints')
)

for n in n_range:
    results_df = pd.read_csv(
        get_or_create_output_directory(n).joinpath('visibility.csv'),
        index_col=0,
        header=[0,1]
    )
    scores.loc[n] = get_scores(gt_df, results_df, cols)
    print(n, *get_scores(gt_df, results_df, cols))
    

2 0.7608861726508785 0.492 0.39805825242718446 0.4400715563506261
4 0.8792971734148205 0.7559322033898305 0.7216828478964401 0.7384105960264901
8 0.9423223834988541 0.8688783570300158 0.889967637540453 0.8792965627498002
16 0.9648586707410237 0.9071207430340558 0.948220064724919 0.9272151898734177
32 0.970970206264324 0.9156441717791411 0.9660194174757282 0.9401574803149606


In [20]:
scores

,Accuracy,Precision,Recall,F1-Score
Number of viewpoints,,,,
2,0.760886,0.492000,0.398058,0.440072
4,0.879297,0.755932,0.721683,0.738411
8,0.942322,0.868878,0.889968,0.879297
16,0.964859,0.907121,0.948220,0.927215
32,0.970970,0.915644,0.966019,0.940157


In [21]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=scores.index, y=scores['Accuracy'], name='Accuracy'))
fig.add_trace(go.Scatter(x=scores.index, y=scores['Precision'], name='Precision'))
fig.add_trace(go.Scatter(x=scores.index, y=scores['Recall'], name='Recall'))
fig.add_trace(go.Scatter(x=scores.index, y=scores['F1-Score'], name='F1-Score'))
fig.update_layout(
    template='plotly_white', 
    yaxis=dict(title='Value', range=[0,1]),
)
fig.update_xaxes(type="log", title='Number of viewpoints')
fig.write_image(FIGURES_DIR.joinpath('performance.png'), scale=3)
fig.show()